In [25]:
import yfinance as yf
import pandas as pd
import datetime
from fredapi import Fred
import numpy as np
import pandas_ta as ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import load_model
import tensorflow as tf
tf.get_logger().setLevel('ERROR')  # Options: 'DEBUG', 'INFO', 'WARNING', 'ERROR', 'FATAL'


In [26]:
def prepare_inference_data(company_df, sequence_length=60):
    """
    Prepare input data for inference .
    Args:
        company_df (DataFrame): The DataFrame for a specific company.
        sequence_length (int): The number of past days to consider as input.

    Returns:
        numpy array: The input data ready for prediction.
    """
    # Ensure data is sorted by date
    company_df = company_df.sort_index()

    # Select relevant input features (exclude targets)
    input_features = company_df.filter(regex="^(?!.*target).*").values

    # Take the last `sequence_length` days as input for prediction
    if len(input_features) >= sequence_length:
        input_sequence = input_features[-sequence_length:]
        return np.expand_dims(input_sequence, axis=0)  # Add batch dimension
    else:
        raise ValueError("Insufficient data for inference (less than sequence length).")

In [27]:
def get_predictions_for_all_companies(all_dfs, model, ticker, sequence_length=60):
    """
    Get predictions for all companies using the trained model.
    Args:
        all_dfs (dict): Dictionary of company DataFrames.
        model: Trained LSTM model.
        sequence_length (int): Number of past days to consider as input.

    Returns:
        dict: Predictions for each company.
    """
    predictions = {}

    all_dfs = {k: v for k, v in all_dfs.items() if k == "df_"+ticker}
    
    for company, df in all_dfs.items():
        try:
            # Prepare data for inference
            input_data = prepare_inference_data(df, sequence_length=sequence_length)
            
            # Make predictions
            pred = model.predict(input_data)
            
            # Store predictions
            predictions[company] = pred.flatten()  # Flatten the array for readability
        
        except ValueError as e:
            print(f"Skipping {company}: {e}")

    return predictions

In [28]:
def data_loading():
    start_date = (datetime.datetime.today() - datetime.timedelta(days=100)).strftime('%Y-%m-%d')
    end_date = datetime.datetime.today().strftime('%Y-%m-%d')

    # Example list of S&P 500 tickers (full list can be obtained elsewhere)
    tickers = ["AAPL", "NVDA", "MSFT", "GOOG", "GOOGL", "AMZN", "META", "AVGO", "LLY", "TSLA", 
                    "WMT", "JPM", "V", "XOM", "UNH", "ORCL", "MA", "HD", "PG", "COST", "JNJ", 
                    "NFLX", "ABBV", "BAC", "KO", "CRM", "CVX", "MRK", "TMUS", "AMD", "PEP", 
                    "ACN", "LIN", "TMO", "MCD", "CSCO", "ADBE", "WFC", "IBM", "GE", "ABT", 
                    "DHR", "AXP", "MS", "CAT", "NOW", "QCOM", "PM", "ISRG", "VZ"]

    # Download data for all tickers at once
    stock_price_data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

    stock_data = stock_price_data.copy()

    # Flatten MultiIndex columns in stock_data
    stock_data.columns = ['_'.join(col).strip() for col in stock_data.columns.values]

    # Convert index to DatetimeIndex if not already
    if not isinstance(stock_data.index, pd.DatetimeIndex):
        stock_data.index = pd.to_datetime(stock_data.index)

    # Sort by date
    stock_data.sort_index(inplace=True)

    with open('updated_all_dfs.pkl', 'rb') as file:
        loaded_data = pickle.load(file)
    
    inference_data = {}

    for comp in tickers:
        ll = 'df_'+comp
        reg = '^'+comp+'_'
        filtered_data = stock_data.filter(regex=reg)
        last_rows = loaded_data[ll].tail(filtered_data.shape[0])
        ind_ = filtered_data.index
        last_rows.index = ind_
        merged_df = filtered_data.join(last_rows, how='left', lsuffix='', rsuffix='_r')
        merged_df = merged_df[[col for col in merged_df.columns if not col.endswith('_r')]]
        inference_data[ll] = merged_df
            
    return inference_data

In [29]:
def lstm_predictions(ticker):
    
    inference_data = data_loading()
    print(inference_data.keys())
    print("Loading data Completed")    
    new_model = load_model('lstm_model_general.h5')
    print("Loading LSTM model Completed")
    predictions = get_predictions_for_all_companies(inference_data, new_model,ticker, sequence_length=60)
    
    return predictions

In [30]:
def main():
    lstm_preds = lstm_predictions("AMZN")
    return lstm_preds

In [31]:
vals = main()

[                       0%                       ]

[*********************100%***********************]  50 of 50 completed
2024-12-12 16:06:46.785450: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-12 16:06:46.786247: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]


dict_keys(['df_AAPL', 'df_NVDA', 'df_MSFT', 'df_GOOG', 'df_GOOGL', 'df_AMZN', 'df_META', 'df_AVGO', 'df_LLY', 'df_TSLA', 'df_WMT', 'df_JPM', 'df_V', 'df_XOM', 'df_UNH', 'df_ORCL', 'df_MA', 'df_HD', 'df_PG', 'df_COST', 'df_JNJ', 'df_NFLX', 'df_ABBV', 'df_BAC', 'df_KO', 'df_CRM', 'df_CVX', 'df_MRK', 'df_TMUS', 'df_AMD', 'df_PEP', 'df_ACN', 'df_LIN', 'df_TMO', 'df_MCD', 'df_CSCO', 'df_ADBE', 'df_WFC', 'df_IBM', 'df_GE', 'df_ABT', 'df_DHR', 'df_AXP', 'df_MS', 'df_CAT', 'df_NOW', 'df_QCOM', 'df_PM', 'df_ISRG', 'df_VZ'])
Loading data Completed


2024-12-12 16:06:46.786861: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_1_grad/concat/split_1/split_dim' with dtype int32
	 [[{{node gradients/split_1_grad/concat/split_1/split_dim}}]]
2024-12-12 16:06:46.887176: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-12 16:06:46.887852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT:

Loading LSTM model Completed


2024-12-12 16:06:47.064281: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-12 16:06:47.065140: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-12 16:06:47.065987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 445ms/step


In [32]:
vals.keys()

dict_keys(['df_AMZN'])

In [34]:
vals['df_AMZN']

array([56.801254, 56.732742, 56.705776, 56.67417 , 56.811554, 56.710457,
       56.703117, 56.638515, 56.915035, 56.75484 , 56.550545, 56.290154,
       56.596928, 56.83077 , 56.909447, 56.971436, 56.834164, 56.673832,
       57.11828 , 56.7683  , 56.512383, 56.873047, 56.874454, 56.97342 ,
       56.850136], dtype=float32)